{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Informe de Mejoras del Proyecto hierarchical-SAE\n",
    "\n",
    "**📅 Fecha:** 18 de septiembre de 2025\n",
    "\n",
    "**🧑‍💻 Autor:** GitHub Copilot\n",
    "\n",
    "## 📝 Resumen Ejecutivo\n",
    "\n",
    "Este documento detalla las correcciones y mejoras implementadas en el proyecto **hierarchical-SAE**. Las modificaciones se centraron en resolver problemas de importación, corregir errores de código y mejorar la robustez general del sistema, sin alterar la funcionalidad principal del proyecto."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔄 1. Gestión Robusta de Importaciones de `quartopy`\n",
    "\n",
    "### 🚨 Problema Identificado\n",
    "\n",
    "Múltiples archivos del proyecto enfrentaban errores al importar el módulo `quartopy`, componente esencial para el funcionamiento del sistema. Las importaciones fallaban cuando el módulo no estaba disponible inmediatamente, sin proporcionar un mecanismo de recuperación adecuado o mensajes de error útiles."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### ✅ Solución Implementada\n",
    "\n",
    "Se diseñó e implementó una función de importación robusta con las siguientes características:\n",
    "\n",
    "- **Intento principal**: Primero intenta importar `quartopy` directamente\n",
    "\n",
    "- **Recuperación automática**: Si falla, utiliza `setup_dependencies.py` para configurar las dependencias\n",
    "\n",
    "- **Mensajes informativos**: Proporciona retroalimentación clara en cada paso del proceso\n",
    "\n",
    "- **Manejo de errores**: Gestiona adecuadamente las excepciones con mensajes útiles\n",
    "\n",
    "- **Validación de ruta**: Asegura que las rutas de importación sean correctas\n",
    "\n",
    "A continuación se muestra la implementación de esta función:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def _validate_and_import_quartopy():\n",
    "    \"\"\"\n",
    "    Validates and imports quartopy dependencies with clear error messages.\n",
    "\n",
    "    Returns:\n",
    "        tuple: (BotAI, Piece, QuartoGame) classes from quartopy\n",
    "\n",
    "    Raises:\n",
    "        ImportError: If quartopy cannot be imported with helpful instructions\n",
    "    \"\"\"\n",
    "    try:\n",
    "        from quartopy import BotAI, Piece, QuartoGame\n",
    "        utils_logger.debug(\"✅ Quartopy importado correctamente\")\n",
    "        return BotAI, Piece, QuartoGame\n",
    "\n",
    "    except ImportError as initial_error:\n",
    "        utils_logger.warning(\"⚠️ Error al importar quartopy, intentando configurar dependencias...\")\n",
    "\n",
    "        # Attempt fallback with setup_dependencies\n",
    "        try:\n",
    "            import sys\n",
    "            from pathlib import Path\n",
    "\n",
    "            # Add parent directory to path for setup_dependencies\n",
    "            parent_dir = Path(__file__).parent.parent\n",
    "            if str(parent_dir) not in sys.path:\n",
    "                sys.path.insert(0, str(parent_dir))\n",
    "\n",
    "            # Import and run dependency setup\n",
    "            import setup_dependencies\n",
    "            setup_dependencies.setup_quartopy(silent=False)\n",
    "\n",
    "            # Retry import after setup\n",
    "            from quartopy import BotAI, Piece, QuartoGame\n",
    "            utils_logger.info(\"✅ Quartopy importado correctamente después de configurar dependencias\")\n",
    "            return BotAI, Piece, QuartoGame\n",
    "\n",
    "        except ImportError as final_error:\n",
    "            error_msg = (\n",
    "                \"❌ ERROR DE DEPENDENCIA: No se puede importar quartopy. \"\n",
    "                \"Asegúrese de que quartopy esté correctamente instalado.\"\n",
    "            )\n",
    "            utils_logger.error(error_msg)\n",
    "            raise ImportError(error_msg) from final_error"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 📁 Archivos Mejorados\n",
    "\n",
    "Esta solución se implementó en los siguientes archivos clave del proyecto:\n",
    "\n",
    "| Archivo | Funcionalidad |\n",
    "|---------|---------------|\n",
    "| `bot/random_bot.py` | Bot de selección aleatoria para el juego Quarto |\n",
    "| `bot/human.py` | Interfaz para jugador humano |\n",
    "| `QuartoRL/RL_functions.py` | Funciones de aprendizaje por refuerzo |\n",
    "| `QuartoRL/contest.py` | Gestión de competiciones entre bots |\n",
    "| `tools/view_training.py` | Visualización de resultados de entrenamiento |\n",
    "\n",
    "En cada archivo, la función se adaptó específicamente para importar los componentes necesarios de `quartopy` según los requerimientos particulares."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🛠️ 2. Corrección de Errores en Archivos de Ejemplo\n",
    "\n",
    "### 2.1. Archivo `a.py` - Creación correcta de Tensores PyTorch\n",
    "\n",
    "#### 🚨 Problema Identificado\n",
    "\n",
    "El archivo utilizaba una sintaxis incorrecta para crear tensores de PyTorch, intentando usar el constructor `Tensor` con argumentos nombrados que no son soportados por la API.\n",
    "\n",
    "```python\n",
    "# Código con error\n",
    "Tensor(shape=torch.Size([200, 1]), device=torch.device(\"cpu\"), dtype=torch.float32, is_shared=False)\n",
    "```\n",
    "\n",
    "#### ✅ Solución Implementada\n",
    "\n",
    "Se modificó el código para utilizar las funciones de fábrica de PyTorch que son el método recomendado para crear tensores:\n",
    "\n",
    "```python\n",
    "# Código corregido\n",
    "torch.zeros(200, 1, dtype=torch.float32)\n",
    "```\n",
    "\n",
    "Además, se simplificó la estructura general del TensorDict para mejorar la legibilidad y mantenibilidad."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.2. Archivo `actions.py` - Parámetros correctos para OneHot\n",
    "\n",
    "#### 🚨 Problema Identificado\n",
    "\n",
    "El archivo contenía un error en la definición de objetos `OneHot` de torchrl, donde el parámetro `shape` no coincidía con el valor de `n`, causando una excepción al ejecutar el código.\n",
    "\n",
    "#### ✅ Solución Implementada\n",
    "\n",
    "Se corrigió la definición para que el último valor de `shape` coincida con `n` como requiere la API de torchrl:\n",
    "\n",
    "```python\n",
    "# Antes (con error)\n",
    "\"piece\": OneHot(n=16, shape=(1,))\n",
    "\n",
    "# Después (corregido)\n",
    "\"piece\": OneHot(n=16, shape=(16,))\n",
    "```"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.3. Archivo `debugging.py` - Actualización de rutas de archivo\n",
    "\n",
    "#### 🚨 Problema Identificado\n",
    "\n",
    "El archivo contenía rutas absolutas que no correspondían con la estructura de directorios del usuario actual, causando errores `FileNotFoundError` al intentar acceder a archivos inexistentes.\n",
    "\n",
    "#### ✅ Solución Implementada\n",
    "\n",
    "Se actualizaron las rutas para reflejar la estructura de directorios correcta del usuario actual:\n",
    "\n",
    "```python\n",
    "# Ruta actualizada a un archivo CSV existente\n",
    "a, b = process_match(\n",
    "    \"C:/Users/bravo/Documents/Metodos Numericos Pycharm/Mech Interp/hierarchical-SAE/partidas_guardadas/ba_increasing_n_last_states/ba_increasing_n_last_states_epoch_0001/2025-09-14_12-55-49_match001.csv\",\n",
    "    result=0,\n",
    ")\n",
    "```"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.4. Archivo `try_collector.py` - Modernización completa\n",
    "\n",
    "#### 🚨 Problemas Identificados\n",
    "\n",
    "El archivo presentaba múltiples problemas críticos:\n",
    "\n",
    "1. Referencias a funciones no definidas (`your_env_fn` y `your_policy`)\n",
    "\n",
    "2. Uso de la biblioteca Gym obsoleta en lugar de su sucesor Gymnasium\n",
    "\n",
    "3. Incompatibilidad al intentar usar `GymEnv` con un entorno personalizado\n",
    "\n",
    "4. Métodos `reset` y `step` no conformes con la API actual\n",
    "\n",
    "#### ✅ Soluciones Implementadas\n",
    "\n",
    "Se realizó una reescritura significativa del archivo:\n",
    "\n",
    "1. **Implementación de funciones faltantes**\n",
    "\n",
    "   ```python\n",
    "   def your_env_fn():\n",
    "       env = CustomEnv()\n",
    "       return GymWrapper(env)\n",
    "   \n",
    "   def your_policy(tensordict):\n",
    "       # Política que toma acciones aleatorias\n",
    "       batch_size = tensordict.batch_size\n",
    "       action = torch.randint(0, 4, batch_size if len(batch_size) > 0 else (1,))\n",
    "       return TensorDict({\"action\": action}, batch_size=batch_size)\n",
    "   ```\n",
    "\n",
    "2. **Actualización a Gymnasium**\n",
    "\n",
    "   ```python\n",
    "   import gymnasium as gym\n",
    "   from gymnasium import spaces\n",
    "   ```\n",
    "\n",
    "3. **Uso correcto de wrappers**\n",
    "\n",
    "   ```python\n",
    "   # En lugar de usar GymEnv con un ID\n",
    "   torchrl_env = GymWrapper(env)  # Envuelve directamente la instancia\n",
    "   ```\n",
    "\n",
    "4. **Actualización de métodos de entorno**\n",
    "\n",
    "   ```python\n",
    "   def reset(self, *, seed=None, options=None):\n",
    "       # API actual de Gymnasium\n",
    "       super().reset(seed=seed)\n",
    "       self.state = np.random.uniform(-0.1, 0.1, size=3).astype(np.float32)\n",
    "       return self.state, {}  # Retorna (observation, info)\n",
    "   \n",
    "   def step(self, action):\n",
    "       # API actual con 5 valores de retorno\n",
    "       return self.state, reward, terminated, truncated, info\n",
    "   ```"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔧 3. Corrección de Errores en `view_training.py`\n",
    "\n",
    "### 🚨 Problema Identificado\n",
    "\n",
    "El archivo presentaba un error al manejar excepciones de la biblioteca `docopt`. Intentaba acceder a `docopt.DocoptExit` cuando `docopt` se había importado como una función, no como un módulo.\n",
    "\n",
    "### ✅ Solución Implementada\n",
    "\n",
    "Se corrigió la importación y el manejo de excepciones:\n",
    "\n",
    "```python\n",
    "# Antes (con error)\n",
    "from docopt import docopt\n",
    "# ...\n",
    "except docopt.DocoptExit:  # Error: docopt es una función, no un módulo\n",
    "    # código de manejo\n",
    "\n",
    "# Después (corregido)\n",
    "from docopt import docopt, DocoptExit\n",
    "# ...\n",
    "except DocoptExit:  # Correcto: importación directa de la excepción\n",
    "    # código de manejo\n",
    "```\n",
    "\n",
    "Adicionalmente, se mejoró la función principal para incluir:\n",
    "\n",
    "- Verificación de existencia de archivos\n",
    "- Mensajes de error más informativos\n",
    "- Códigos de salida apropiados"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🌟 4. Mejoras Globales del Proyecto\n",
    "\n",
    "| Categoría | Descripción | Beneficio |\n",
    "|-----------|-------------|----------|\n",
    "| **🔄 Importaciones robustas** | Sistema que intenta importar, configura dependencias si falla, y proporciona retroalimentación | Reducción de errores de importación, experiencia de usuario mejorada |\n",
    "| **📝 Mensajes mejorados** | Mensajes de error y advertencia más claros e informativos | Facilita la depuración y solución de problemas |\n",
    "| **🔄 Modernización de código** | Actualización de APIs obsoletas (Gym → Gymnasium) | Compatibilidad con versiones actuales de bibliotecas |\n",
    "| **📊 Manejo de loggers** | Estandarización del uso de loggers para evitar conflictos | Consistencia en los registros y mensajes |\n",
    "| **🔍 Validación de rutas** | Corrección de rutas absolutas para adaptarlas al sistema de archivos actual | Prevención de errores FileNotFoundError |"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 5. Conclusiones y Recomendaciones\n",
    "\n",
    "### 📊 Impacto de las Mejoras\n",
    "\n",
    "Las modificaciones implementadas han aumentado significativamente la **robustez** y **mantenibilidad** del proyecto hierarchical-SAE. El sistema ahora:\n",
    "\n",
    "- ✅ Maneja errores de manera más elegante\n",
    "\n",
    "- ✅ Proporciona mensajes útiles para el usuario\n",
    "\n",
    "- ✅ Utiliza APIs modernas y prácticas recomendadas\n",
    "\n",
    "- ✅ Es compatible con diferentes configuraciones de sistema\n",
    "\n",
    "### 🔮 Recomendaciones Futuras\n",
    "\n",
    "Para continuar mejorando el proyecto, se sugiere considerar:\n",
    "\n",
    "1. **Implementar pruebas automatizadas** para verificar el correcto funcionamiento de la importación de dependencias\n",
    "\n",
    "2. **Documentar el procedimiento de instalación** de quartopy para nuevos usuarios\n",
    "\n",
    "3. **Revisar el resto de los archivos** del proyecto para aplicar patrones similares de manejo robusto\n",
    "\n",
    "4. **Considerar un sistema de gestión de dependencias** más moderno como Poetry o Conda\n",
    "\n",
    "### 🎯 Nota Final\n",
    "\n",
    "Las correcciones realizadas mantienen intacta la funcionalidad principal del proyecto mientras mejoran significativamente su usabilidad, lo que facilitará su adopción y extensión por parte de nuevos desarrolladores."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
